In [1]:
#Installation des dépendances en amont si non-présentes dans l'environnement
!pip install numpy pandas scikit-learn tensorflow imblearn openpyxl

In [2]:
#Importation des librairies communes (peu prendre du temps)
import os
import sys
import numpy
import pandas as pd

In [3]:
#Chemin menant aux données
data_path = os.path.join(os.getcwd(), "source")
#création du dictionnaire
data_dict = dict()

In [4]:
# Boucle pour récupérer tous les fichiers CSV dans le dossier et les ajouter au dictionnaire
for file in os.listdir(data_path):
    if file.endswith(".csv"):
        file_path = os.path.join(data_path, file)
        dict_key = file.split('.')[0]  # Nom du fichier sans l'extension .csv
        data_dict[dict_key] = pd.read_csv(file_path)
data_dict.keys()

dict_keys(['appearances', 'club_games', 'clubs', 'competitions', 'game_events', 'game_lineups', 'games', 'player_valuations', 'players', 'clubs_link_api'])

In [5]:
# Importation des bibliothèques nécessaires
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [6]:
df_games = data_dict["games"]
df_games.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url',
       'home_club_formation', 'away_club_formation', 'home_club_name',
       'away_club_name', 'aggregate', 'competition_type'],
      dtype='object')

In [7]:
# Fonction pour déterminer le résultat
def determine_result(row):
    if row['home_club_goals'] > row['away_club_goals']:
        return 1
    elif row['home_club_goals'] < row['away_club_goals']:
        return -1
    else:
        return 0

# Ajout de la colonne 'resultat'
df_games['resultat'] = df_games.apply(determine_result, axis=1)
df_games.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url',
       'home_club_formation', 'away_club_formation', 'home_club_name',
       'away_club_name', 'aggregate', 'competition_type', 'resultat'],
      dtype='object')

In [8]:
#Toutes les fonctions
def games_of_the_season(df_games, club_id, saison, lieu='tous'):
    """
    Récupère tous les matchs d'une saison pour un club spécifié, ou les 10 derniers matchs si moins de 5 matchs dans la saison.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    saison (str) : Saison à filtrer.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Filtrage initial par saison et lieu
    if lieu == 'domicile':
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) & (df_games['season'] == saison)]
    elif lieu == 'exterieur':
        matchs_du_club = df_games[(df_games['away_club_id'] == club_id) & (df_games['season'] == saison)]
    else:
        matchs_du_club = df_games[((df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)) & (df_games['season'] == saison)]
    
    # Vérification du nombre de matchs
    if len(matchs_du_club) < 5:
        # Prendre les 10 derniers matchs toutes saisons confondues si moins de 5 matchs dans la saison
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(10)
    else:
        # Trier par date
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'season', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]

def last_five_games(df_games, club_id, date, lieu='tous'):
    """
    Récupère les 5 derniers matchs d'un club spécifié avant une date donnée en fonction du lieu du match.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    date (str ou pd.Timestamp) : Date avant laquelle les matchs doivent être récupérés.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Assurer que la colonne 'date' est au format datetime
    df_games['date'] = pd.to_datetime(df_games['date'])

    # Filtrage des matchs avant la date donnée
    matchs_du_club = df_games[df_games['date'] < date]
    
    if lieu == 'domicile':
        matchs_du_club = matchs_du_club[matchs_du_club['home_club_id'] == club_id]
    elif lieu == 'exterieur':
        matchs_du_club = matchs_du_club[matchs_du_club['away_club_id'] == club_id]
    else:  # 'tous'
        matchs_du_club = matchs_du_club[(matchs_du_club['home_club_id'] == club_id) | (matchs_du_club['away_club_id'] == club_id)]
    
    # Tri des matchs par date et sélection des 5 derniers
    matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(5)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]
    
def last_game_id(df_games, club_id):
    """
    Récupère l'ID du dernier match joué par un club spécifié.

    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.

    Retourne:
    int : ID du dernier match joué par le club.
    """
    # Filtrer les matchs impliquant le club
    matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
    
    # Trier par date en ordre décroissant et prendre le premier match
    dernier_match = matchs_du_club.sort_values(by='date', ascending=False).iloc[0]

    return dernier_match['game_id']

def match_sheet(df_game_lineups, match_id, club_id):
    """
    Récupère tous les player_id qui ont participé à un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs qui ont participé au match.
    """
    # Filtrer pour obtenir les lignes correspondant au match_id et club_id spécifiés
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & (df_game_lineups['club_id'] == club_id)]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

def starting_lineup_match(df_game_lineups, match_id, club_id):
    """
    Récupère les player_id des joueurs qui étaient dans la composition de départ d'un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs de la composition de départ.
    """
    #df_game_lineups = data_dict['game_lineups']
    # Filtrer pour obtenir les lignes correspondant au match_id, club_id et type 'starting_lineup'
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & 
                                      (df_game_lineups['club_id'] == club_id) & 
                                      (df_game_lineups['type'] == 'starting_lineup')]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

def market_value_player(df_player, player_id):
    """
    Récupère la valeur marchande d'un joueur spécifié.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    player_id (int) : ID du joueur.
    
    Retourne:
    float : La valeur marchande du joueur en euros.
    """
    # Trouver l'entrée correspondant au player_id
    valeur_joueur = df_player[df_player['player_id'] == player_id]['market_value_in_eur']

    # Retourner la valeur marchande, ou None si le joueur n'est pas trouvé
    return valeur_joueur.iloc[0] if not valeur_joueur.empty else None

def mean_team_value(df_player, liste_player_id):
    """
    Calcule la valeur marchande moyenne d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Valeur marchande moyenne de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs) / len(valeurs) if valeurs else 0

def sum_team_value(df_player, liste_player_id):
    """
    Calcule la somme des valeurs marchandes d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Somme des valeurs marchandes de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs)




In [9]:
df_games.head()
df_games['date'] = pd.to_datetime(df_games['date'])
df_games = df_games.sort_values(by='date', ascending=True)

In [10]:
# Supposons que df_games est votre DataFrame original
selected_columns = ["game_id", "season", "date", "home_club_id", "away_club_id", "home_club_position", "away_club_position", "resultat"]
# Création du nouveau DataFrame avec seulement les colonnes sélectionnées
df_final = df_games[selected_columns].dropna()
df_final = df_final[(df_final["season"] == 2020) | (df_final["season"] == 2021) | (df_final["season"] == 2022) | (df_final["season"] == 2023)]
df_final.head(5)

,game_id,season,date,home_club_id,away_club_id,home_club_position,away_club_position,resultat
13339,3393087,2020,2020-08-01,903,2553,2.0,8.0,1
20734,3393086,2020,2020-08-01,1519,2578,6.0,7.0,0
17040,3393088,2020,2020-08-01,465,1241,4.0,11.0,1
2591,3393089,2020,2020-08-01,370,124,9.0,3.0,-1
33909,3393090,2020,2020-08-02,371,2999,1.0,12.0,1


In [11]:
df_players = data_dict['players']  # Assurez-vous d'avoir le DataFrame correct pour les évaluations des joueurs
df_game_lineups = data_dict['game_lineups']

In [ ]:
# Ajout de la valeur marchande totale des équipes
df_final['home_team_market_value'] = df_final.apply(lambda row: sum_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, row['home_club_id']), row['home_club_id'])), axis=1)
df_final['away_team_market_value'] = df_final.apply(lambda row: sum_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, row['away_club_id']), row['away_club_id'])), axis=1)
df_final.head()

In [ ]:
def calculer_forme(df_games, club_id, date_du_match, lieu='tous'):
    """
    Calcule la forme d'un club sur ses 5 derniers matchs avant la date du match spécifiée.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    date_du_match (str ou pd.Timestamp) : Date du match pour lequel la forme est calculée.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    int : Les points accumulés par le club sur ses 5 derniers matchs.
    """
    # Assurer que la colonne 'date' est au format datetime
    #df_games['date'] = pd.to_datetime(df_games['date'])

    # Filtrer les matchs précédents et trier par date
    matchs_precedents = last_five_games(df_games, club_id, date_du_match, lieu)
    
    # Calculer les points (exemple simplifié)
    points = 0
    for _, row in matchs_precedents.iterrows():
        if row['home_club_id'] == club_id:
            points += 3 if row['home_club_goals'] > row['away_club_goals'] else 1 if row['home_club_goals'] == row['away_club_goals'] else 0
        else:
            points += 3 if row['away_club_goals'] > row['home_club_goals'] else 1 if row['away_club_goals'] == row['home_club_goals'] else 0
    
    return points


In [ ]:
# Ajout de la forme des équipes
df_final['home_team_form'] = df_final.apply(lambda x: calculer_forme(df_games, x["home_club_id"], x["date"], "domicile"), axis=1)
df_final['away_team_form'] = df_final.apply(lambda x: calculer_forme(df_games, x["away_club_id"],  x["date"], "exterieur"), axis=1)
df_final.head()

In [ ]:
print(f"Taille du df {len(df_to_train)}")
print("Nombre de lignes avec 0 dans 'resultat':", (df_to_train['resultat'] == 0).sum())
print("Nombre de lignes avec -1 dans 'resultat':", (df_to_train['resultat'] == -1).sum())
print("Nombre de lignes avec 1 dans 'resultat':", (df_to_train['resultat'] == 1).sum())
print("PB home market value:", (df_to_train['home_team_market_value'] < 1).sum())
print("PB away market value:", (df_to_train['away_team_market_value'] < 1).sum())
print("PB home market value:", (df_to_train['home_team_form'] < 1).sum())
print("PB away market value:",  (df_to_train['away_team_form'] < 1).sum())

In [ ]:
# Suppression des lignes où 'home_team_market_value' < 1
df_to_train = df_to_train.loc[df_final['home_team_market_value'] >= 1]
# Suppression des lignes où 'away_team_market_value' < 1
df_to_train = df_to_train.loc[df_final['away_team_market_value'] >= 1]
# Suppression des lignes où 'home_team_form' < 1
df_to_train = df_to_train.loc[df_final['home_team_form'] >= 1]
# Suppression des lignes où 'away_team_form' < 1
df_to_train = df_to_train.loc[df_final['away_team_form'] >= 1]
df_to_train.head()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.utils import to_categorical

# Supposons que df_final est déjà défini et contient les données nécessaires

# 1. Préparation et nettoyage des données
conditions = (
    (df_final['home_team_market_value'] >= 1) & 
    (df_final['away_team_market_value'] >= 1) & 
    (df_final['home_team_form'] >= 1) & 
    (df_final['away_team_form'] >= 1)
)
df_to_train = df_final[conditions]
df_to_train = df_to_train.drop(["season", "game_id", "away_club_position", "home_club_position", "date"], axis=1).dropna()

import openpyxl
df_final.to_excel("df_final.xlsx", index=False)
df_to_train.to_excel("df_to_train.xlsx", index=False)

df_to_train.head(10)

In [ ]:
print(f"Taille du df {len(df_to_train)}")
print("Nombre de lignes avec 0 dans 'resultat':", (df_to_train['resultat'] == 0).sum())
print("Nombre de lignes avec -1 dans 'resultat':", (df_to_train['resultat'] == -1).sum())
print("Nombre de lignes avec 1 dans 'resultat':", (df_to_train['resultat'] == 1).sum())
print("PB home market value:", (df_to_train['home_team_market_value'] < 1).sum())
print("PB away market value:", (df_to_train['away_team_market_value'] < 1).sum())
print("PB home market value:", (df_to_train['home_team_form'] < 1).sum())
print("PB away market value:",  (df_to_train['away_team_form'] < 1).sum())